In [1]:
import sys
sys.path.insert(0, '..')

from multipliers import (
    NaiveMultiplier, 
    RSRBinaryMultiplier, 
    RSRTernaryMultiplier,
    RSRPlusPlusBinaryMultiplier,
    RSRPlusPlusTernaryMultiplier
)

import math
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator

import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
def generate_random_binary_matrix(n):
    # Generate a random binary matrix of size n x n
    binary_matrix = np.random.randint(2, size=(n, n))
    return binary_matrix

def generate_random_ternary_matrix(n):
    ternary_matrix = np.random.randint(low=-1, high=2, size=(n, n))
    return ternary_matrix

def generate_random_int_vector(size, low=0, high=100):
    random_vector = np.random.randint(low, high, size)
    return random_vector

In [8]:
n_s = [2**i for i in range(11, 17)]

In [9]:
vectors = [generate_random_int_vector(n) for n in n_s]
binary_matrices = [generate_random_binary_matrix(n) for n in n_s]
#ternary_matrices = [generate_random_ternary_matrix(n) for n in n_s]

In [10]:
k_optimals = {
    "rsrpp" : [5, 6, 8, 8, 9, 10],
    "rsr": [4, 4, 5, 6, 6, 6]
}

## Binary

In [11]:
naive_times = []
rsr_times = []
rsrpp_times = []


    
for i, (v, A) in enumerate(zip(vectors, binary_matrices)):
    print(f"k: {k_optimals.get("rsr")[i]} | {i + 1} / {len(n_s)}")
    rsr = RSRBinaryMultiplier(A, k=k_optimals.get("rsr")[i])

    print("rsr: ", end="")
    rsr_time = %timeit -r 4 -o rsr.multiply(v)  
    rsr_times.append(rsr_time.best)  

for i, (v, A) in enumerate(zip(vectors, binary_matrices)):
    print(f"k: {k_optimals.get("rsrpp")[i]} | {i + 1} / {len(n_s)}")
    rsrpp = RSRPlusPlusBinaryMultiplier(A, k=k_optimals.get("rsrpp")[i])

    print("rsrpp: ", end="")
    rsrpp_time = %timeit -r 4 -o rsrpp.multiply(v)
    rsrpp_times.append(rsrpp_time.best)


for i, (v, A) in enumerate(zip(vectors, binary_matrices)):
    naive = NaiveMultiplier(A)
    
    print("naive: ", end="")
    
    naive_time = %timeit -r 4 -o naive.multiply(v)
    naive_times.append(naive_time.best) 

naive_times = np.array(naive_times)
rsr_times = np.array(rsr_times)
rsrpp_times = np.array(rsrpp_times)

for length, naive_time, rsr_time, rsrpp_time in zip([len(v) for v in vectors], naive_times, rsr_times, rsrpp_times):
    print(f"Vector length: {length} | Naive time: {naive_time:.6f} seconds | RSR time: {rsr_time:.6f} seconds | RSRPP time: {rsrpp_time:.6f}")


k: 4 | 1 / 2
rsr: 13.5 ms ± 42.7 μs per loop (mean ± std. dev. of 4 runs, 100 loops each)
k: 4 | 2 / 2
rsr: 33.7 ms ± 151 μs per loop (mean ± std. dev. of 4 runs, 10 loops each)
k: 5 | 1 / 2
rsrpp: 19.1 ms ± 56.5 μs per loop (mean ± std. dev. of 4 runs, 100 loops each)
k: 6 | 2 / 2
rsrpp: 47.1 ms ± 108 μs per loop (mean ± std. dev. of 4 runs, 10 loops each)
naive: 22.8 ms ± 68.2 μs per loop (mean ± std. dev. of 4 runs, 10 loops each)
naive: 147 ms ± 358 μs per loop (mean ± std. dev. of 4 runs, 10 loops each)
Vector length: 2048 | Naive time: 0.022676 seconds | RSR time: 0.013405 seconds | RSRPP time: 0.019047
Vector length: 4096 | Naive time: 0.146511 seconds | RSR time: 0.033453 seconds | RSRPP time: 0.046914


In [12]:
vector_lengths = np.array(n_s) 

data = {
    'n': vector_lengths,
    'standard': naive_times,
    'rsr': rsr_times,
    'rsrpp': rsrpp_times
}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
output_file = 'reports/numpy_binary_times_2.csv'
df.to_csv(output_file, index=False)

print(f"Execution times saved to {output_file}")


Execution times saved to reports/numpy_binary_times_2.csv


## Ternary

In [5]:
naive_times_ternay = []
rsr_times_ternary = []
rsrpp_times_ternary = []


    
for i, (v, A) in enumerate(zip(vectors, ternary_matrices)):
    k = max(int(math.log2(n_s[i]) - math.log(math.log2(n_s[i]))) - 5, 3)
    print(f"k: {k} | {i + 1} / {len(n_s)}")
    rsr = RSRTernaryMultiplier(A, k=k)

    print("rsr: ", end="")
    rsr_time = %timeit -r 4 -o rsr.multiply(v)  
    rsr_times_ternary.append(rsr_time.best)  

for i, (v, A) in enumerate(zip(vectors, ternary_matrices)):
    k = max(int(math.log2(n_s[i]) - math.log(math.log2(n_s[i]))) - 5, 3)
    print(f"k: {k} | {i + 1} / {len(n_s)}")
    rsrpp = RSRPlusPlusTernaryMultiplier(A, k=k)

    print("rsrpp: ", end="")
    rsrpp_time = %timeit -r 4 -o rsrpp.multiply(v)
    rsrpp_times_ternary.append(rsrpp_time.best)


for i, (v, A) in enumerate(zip(vectors, ternary_matrices)):
    print(f"k: {math.log2(n_s[i])} | {i + 1} / {len(n_s)}")
    naive = NaiveMultiplier(A)
    
    print("naive: ", end="")
    
    naive_time = %timeit -r 4 -o naive.multiply(v)
    naive_times_ternay.append(naive_time.best) 

naive_times_ternay = np.array(naive_times_ternay)
rsr_times_ternary = np.array(rsr_times_ternary)
rsrpp_times_ternary = np.array(rsrpp_times_ternary)

for length, naive_time, rsr_time, rsrpp_time in zip([len(v) for v in vectors], naive_times_ternay, rsr_times_ternary, rsrpp_times_ternary):
    print(f"Vector length: {length} | Naive time: {naive_time:.6f} seconds | RSR time: {rsr_time:.6f} seconds | RSRPP time: {rsrpp_time:.6f}")


k: 3 | 1 / 5
rsr: 27 ms ± 37.1 μs per loop (mean ± std. dev. of 4 runs, 10 loops each)
k: 4 | 2 / 5
rsr: 66.4 ms ± 209 μs per loop (mean ± std. dev. of 4 runs, 10 loops each)
k: 5 | 3 / 5
rsr: 179 ms ± 611 μs per loop (mean ± std. dev. of 4 runs, 10 loops each)
k: 6 | 4 / 5
rsr: 559 ms ± 548 μs per loop (mean ± std. dev. of 4 runs, 1 loop each)
k: 7 | 5 / 5
rsr: 1.9 s ± 3.53 ms per loop (mean ± std. dev. of 4 runs, 1 loop each)
k: 3 | 1 / 5
rsrpp: 44.7 ms ± 98.8 μs per loop (mean ± std. dev. of 4 runs, 10 loops each)
k: 4 | 2 / 5
rsrpp: 91.4 ms ± 401 μs per loop (mean ± std. dev. of 4 runs, 10 loops each)
k: 5 | 3 / 5
rsrpp: 218 ms ± 1.56 ms per loop (mean ± std. dev. of 4 runs, 1 loop each)
k: 6 | 4 / 5
rsrpp: 606 ms ± 2.3 ms per loop (mean ± std. dev. of 4 runs, 1 loop each)
k: 7 | 5 / 5
rsrpp: 1.93 s ± 4.54 ms per loop (mean ± std. dev. of 4 runs, 1 loop each)
k: 11.0 | 1 / 5
naive: 35.6 ms ± 442 μs per loop (mean ± std. dev. of 4 runs, 10 loops each)
k: 12.0 | 2 / 5
naive: 185 ms ±

In [25]:

vector_lengths = np.array(n_s)  # Length of each vector

# speedup_ternary = naive_times_ternary / rsr_times_ternary


# Assuming vector_lengths, naive_times, and rsr_times are defined from the previous code
# Convert data to a DataFrame
data = {
    'Vector Length': vector_lengths,
    'Naive Time (s)': naive_times_ternary,
    'RSR Time (s)': rsr_times_ternary,
    'RSR PP': rsr_pp_times_ternary
}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
output_file = 'multiplication_times_ternary_plus.csv'
df.to_csv(output_file, index=False)

print(f"Execution times saved to {output_file}")


Execution times saved to multiplication_times_ternary_plus.csv
